# CZii 2-Stage model
The first two parts (YOLO & UNET forked from YukiZ [here](https://www.kaggle.com/code/hideyukizushi/czii-yolo11-unet3d-monai-lb-707)

My final strategy:  
- Use a blend of two object detectors (YOLOv11 & UNET) with lowered thresholds to provide regions of interest with high recall but low precision.
- Use the centroid of each ROI, and crop 32x32x32 arrays around it.
- Reshape that array by selecting 12 slices in the z direction, and arranging the arrays 3x2x2
- Now I have a 3x64 'image' that I feed to a mixture of 7 TIMM image backbones.
- 3 x TTA, so final prediction was a mean of 21 slightly different scores.
- A prediction is made, if any 2x detectors agreed with my models above a threshold of 0.4
- Also a prediction is made if 1x detector agreed with my model above a threshold of 0.55
- Models were trained on Sythetic and Real Data, cropped to 40x40x40 around labelled centroids

This wasn't my original goal. I wanted to use Hessian blob detection or some similarly classical approach, then rely only on my own models for classification.  But at this point my models appear to be relatively weak on their own. Ignoring the class predictions of the detectors, my LB scores were only in the 60s.  In my own training I get CV scores with F4 metric in the mid 90s.  So I think the issue must be in generalising to the new data.

# **《《《 YOLO 》》》**

In [1]:
from IPython.display import clear_output
!tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
!pip install --no-index --find-links=./packages ultralytics
!rm -rf ./packages
try:
    import zarr
except: 
    !cp -r '/kaggle/input/hengck-czii-cryo-et-01/wheel_file' '/kaggle/working/'
    !pip install /kaggle/working/wheel_file/asciitree-0.3.3/asciitree-0.3.3
    !pip install --no-index --find-links=/kaggle/working/wheel_file zarr
    !pip install --no-index --find-links=/kaggle/working/wheel_file connected-components-3d
from typing import List, Tuple, Union
deps_path = '/kaggle/input/czii-cryoet-dependencies'
! pip install -q --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt

import sys
sys.path.append('/kaggle/input/hengck-czii-cryo-et-01')
from czii_helper import *
from dataset import *
from model2 import *
clear_output()

In [2]:
import os
import glob
import time
import sys
import warnings
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import cv2
import torch
from tqdm import tqdm
from ultralytics import YOLO
import zarr
from scipy.spatial import cKDTree
from collections import defaultdict
import lightning.pytorch as pl
from datetime import datetime
import pytz

import cc3d
from monai.data import CacheDataset
from monai.transforms import Compose, EnsureType
from torch import nn
from tqdm import tqdm
from monai.networks.nets import UNet

from typing import List, Tuple, Union
from monai.data import DataLoader, Dataset, CacheDataset, decollate_batch
from monai.transforms import (
    Compose, 
    EnsureChannelFirstd, 
    Orientationd,  
    AsDiscrete,  
    RandFlipd, 
    RandRotate90d, 
    NormalizeIntensityd,
    RandCropByLabelClassesd,
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
#Imports for reclassification stage

#Standard Python
import gc
import json
import random
import ast
from datetime import datetime
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from pathlib import Path
import matplotlib.pyplot as plt
import torch.multiprocessing as mp

#Machine Learning
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import average_precision_score
from tqdm import tqdm
from sklearn.cluster import DBSCAN

#PyTorch
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
import timm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint,  EarlyStopping
from pytorch_lightning.loggers import CSVLogger

#MONAI
from monai.transforms import (
    Lambda,
    Compose, 
    OneOf,
    RandRotate90,
    RandFlip,
    RandSpatialCrop,
)

/usr/local/lib/python3.10/dist-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [4]:
model_path = '/kaggle/input/czii-yolo-l-trained-with-synthetic-data/best_synthetic.pt'
model = YOLO(model_path)

In [5]:
runs_path = '/kaggle/input/czii-cryo-et-object-identification/test/static/ExperimentRuns/*'
runs = sorted(glob.glob(runs_path))
runs = [os.path.basename(run) for run in runs]
sp = len(runs)//2
runs1 = runs[:sp]
runs1[:5]

#add by @minfuka
runs2 = runs[sp:]
runs2[:5]

#add by @minfuka
assert torch.cuda.device_count() == 2

In [6]:
particle_names = [
    'apo-ferritin',
    'beta-amylase',
    'beta-galactosidase',
    'ribosome',
    'thyroglobulin',
    'virus-like-particle'
]

particle_to_index = {
    'apo-ferritin': 0,
    'beta-amylase': 1,
    'beta-galactosidase': 2,
    'ribosome': 3,
    'thyroglobulin': 4,
    'virus-like-particle': 5
}

index_to_particle = {index: name for name, index in particle_to_index.items()}

particle_radius = {
    'apo-ferritin': 60,
    'beta-amylase': 65,
    'beta-galactosidase': 90,
    'ribosome': 150,
    'thyroglobulin': 130,
    'virus-like-particle': 135,
}


In [7]:
# add by @sesasj
class UnionFind:
    def __init__(self, size):
        self.parent = np.arange(size)
        self.rank = np.zeros(size, dtype=int)

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])  
        return self.parent[u]

    def union(self, u, v):
        u_root = self.find(u)
        v_root = self.find(v)
        if u_root == v_root:
            return
            
        if self.rank[u_root] < self.rank[v_root]:
            self.parent[u_root] = v_root
        else:
            self.parent[v_root] = u_root
            if self.rank[u_root] == self.rank[v_root]:
                self.rank[u_root] += 1

class PredictionAggregator:
    def __init__(self, first_conf=0.2, conf_coef=0.75):
        self.first_conf = first_conf
        self.conf_coef = conf_coef
        ##############################################################################################################################
        ##############################################################################################################################
        #self.particle_confs = np.array([0.5, 0.0, 0.2, 0.5, 0.2, 0.5])
        self.particle_confs = np.array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3])
        ##############################################################################################################################
        ##############################################################################################################################
        
    def convert_to_8bit(self, volume):
        lower, upper = np.percentile(volume, (0.5, 99.5))
        clipped = np.clip(volume, lower, upper)
        scaled = ((clipped - lower) / (upper - lower + 1e-12) * 255).astype(np.uint8)
        return scaled

    def make_predictions(self, run_id, model, device_no):
        volume_path = f'/kaggle/input/czii-cryo-et-object-identification/test/static/ExperimentRuns/{run_id}/VoxelSpacing10.000/denoised.zarr'
        volume = zarr.open(volume_path, mode='r')[0]
        volume_8bit = self.convert_to_8bit(volume)
        num_slices = volume_8bit.shape[0]

        detections = {
            'particle_type': [],
            'confidence': [],
            'x': [],
            'y': [],
            'z': []
        }

        for slice_idx in range(num_slices):
            
            img = volume_8bit[slice_idx]
            input_image = cv2.resize(np.stack([img]*3, axis=-1), (640, 640))

            results = model.predict(
                input_image,
                save=False,
                imgsz=640,
                conf=self.first_conf,
                device=device_no,
                batch=1,
                verbose=False,
            )

            for result in results:
                boxes = result.boxes
                if boxes is None:
                    continue
                cls = boxes.cls.cpu().numpy().astype(int)
                conf = boxes.conf.cpu().numpy()
                xyxy = boxes.xyxy.cpu().numpy()

                xc = ((xyxy[:, 0] + xyxy[:, 2]) / 2.0) * 10 * (63/64) # 63/64 because of the resize
                yc = ((xyxy[:, 1] + xyxy[:, 3]) / 2.0) * 10 * (63/64)
                zc = np.full(xc.shape, slice_idx * 10 + 5)

                particle_types = [index_to_particle[c] for c in cls]

                detections['particle_type'].extend(particle_types)
                detections['confidence'].extend(conf)
                detections['x'].extend(xc)
                detections['y'].extend(yc)
                detections['z'].extend(zc)

        if not detections['particle_type']:
            return pd.DataFrame()  

        particle_types = np.array(detections['particle_type'])
        confidences = np.array(detections['confidence'])
        xs = np.array(detections['x'])
        ys = np.array(detections['y'])
        zs = np.array(detections['z'])

        aggregated_data = []

        for idx, particle in enumerate(particle_names):
            if particle == 'beta-amylase':
                continue 

            mask = (particle_types == particle)
            if not np.any(mask):
                continue  
                
            particle_confidences = confidences[mask]
            particle_xs = xs[mask]
            particle_ys = ys[mask]
            particle_zs = zs[mask]
            # -------------modified by @sersasj ------------------------
            coords = np.vstack((particle_xs, particle_ys, particle_zs)).T

           
            z_distance = 30 # How many slices can you "jump" to aggregate predictions 10 = 1, 20 = 2...
            xy_distance = 20 # xy_tol_p2 in original code by ITK8191
            
            max_distance = math.sqrt(z_distance**2 + xy_distance**2)
            tree = cKDTree(coords)            
            pairs = tree.query_pairs(r=max_distance, p=2)

            
            uf = UnionFind(len(coords))
            
            coords_xy = coords[:, :2]
            coords_z = coords[:, 2]
            for u, v in pairs:
                z_diff = abs(coords_z[u] - coords_z[v])
                if z_diff > z_distance:
                    continue  

                xy_diff = np.linalg.norm(coords_xy[u] - coords_xy[v])
                if xy_diff > xy_distance:
                    continue  

                uf.union(u, v)

            roots = np.array([uf.find(i) for i in range(len(coords))])
            unique_roots, inverse_indices, counts = np.unique(roots, return_inverse=True, return_counts=True)
            conf_sums = np.bincount(inverse_indices, weights=particle_confidences)
            
            aggregated_confidences = conf_sums / (counts ** self.conf_coef)
            cluster_per_particle = [4,1,2,9,4,8]
            valid_clusters = (counts >= cluster_per_particle[idx]) & (aggregated_confidences > self.particle_confs[idx])

            if not np.any(valid_clusters):
                continue  

            cluster_ids = unique_roots[valid_clusters]

            centers_x = np.bincount(inverse_indices, weights=particle_xs) / counts
            centers_y = np.bincount(inverse_indices, weights=particle_ys) / counts
            centers_z = np.bincount(inverse_indices, weights=particle_zs) / counts

            centers_x = centers_x[valid_clusters]
            centers_y = centers_y[valid_clusters]
            centers_z = centers_z[valid_clusters]

            aggregated_df = pd.DataFrame({
                'experiment': [run_id] * len(centers_x),
                'particle_type': [particle] * len(centers_x),
                'x': centers_x,
                'y': centers_y,
                'z': centers_z
            })

            aggregated_data.append(aggregated_df)

        if aggregated_data:
            return pd.concat(aggregated_data, axis=0)
        else:
            return pd.DataFrame()  

In [8]:
# instance main class
##############################################################################################################################
##############################################################################################################################
#aggregator = PredictionAggregator(first_conf=0.22,  conf_coef=0.39) #Update
aggregator = PredictionAggregator(first_conf=0.17,  conf_coef=0.39) #Olly's Update
##############################################################################################################################
##############################################################################################################################
aggregated_results = []
#add by @minfuka
from concurrent.futures import ProcessPoolExecutor #add by @minfuka

#add by @minfuka
def inference(runs, model, device_no):
    subs = []
    for r in tqdm(runs, total=len(runs)):
        df = aggregator.make_predictions(r, model, device_no)
        subs.append(df)
    
    return subs
start_time = time.time()

with ProcessPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(inference, (runs1, runs2), (model, model), ("0", "1")))


end_time = time.time()

estimated_total_time = (end_time - start_time) / len(runs) * 500  
print(f'estimated total prediction time for 500 runs: {estimated_total_time:.4f} seconds')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 2/2 [00:26<00:00, 13.19s/it]


estimated total prediction time for 500 runs: 4614.8793 seconds


In [9]:
#change by @minfuka
submission0 = pd.concat(results[0])
submission1 = pd.concat(results[1])
submission_ = pd.concat([submission0, submission1]).reset_index(drop=True)

In [10]:
submission_.insert(0, 'id', range(len(submission_)))

In [11]:
submission_.tail()

,id,experiment,particle_type,x,y,z
991,991,TS_6_4,virus-like-particle,4512.710061,5146.912497,1180.0
992,992,TS_6_4,virus-like-particle,5098.582835,4126.212465,1035.0
993,993,TS_6_4,virus-like-particle,1451.382610,4807.155391,1125.0
994,994,TS_6_4,virus-like-particle,3708.760895,5591.139954,1232.5
995,995,TS_6_4,virus-like-particle,2411.612640,5142.742615,1530.0


# **《《《 Unet3D(Monai) 》》》**

In [12]:
class Model(pl.LightningModule):
    def __init__(
        self, 
        spatial_dims: int = 3,
        in_channels: int = 1,
        out_channels: int = 7,
        channels: Union[Tuple[int, ...], List[int]] = (48, 64, 80, 80),
        strides: Union[Tuple[int, ...], List[int]] = (2, 2, 1),
        num_res_units: int = 1,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.model = UNet(
            spatial_dims=self.hparams.spatial_dims,
            in_channels=self.hparams.in_channels,
            out_channels=self.hparams.out_channels,
            channels=self.hparams.channels,
            strides=self.hparams.strides,
            num_res_units=self.hparams.num_res_units,
        )
    def forward(self, x):
        return self.model(x)

channels = (48, 64, 80, 80)
strides_pattern = (2, 2, 1)
num_res_units = 1
def extract_3d_patches_minimal_overlap(arrays: List[np.ndarray], patch_size: int) -> Tuple[List[np.ndarray], List[Tuple[int, int, int]]]:
    if not arrays or not isinstance(arrays, list):
        raise ValueError("Input must be a non-empty list of arrays")
    
    # Verify all arrays have the same shape
    shape = arrays[0].shape
    if not all(arr.shape == shape for arr in arrays):
        raise ValueError("All input arrays must have the same shape")
    
    if patch_size > min(shape):
        raise ValueError(f"patch_size ({patch_size}) must be smaller than smallest dimension {min(shape)}")
    
    m, n, l = shape
    patches = []
    coordinates = []
    
    # Calculate starting positions for each dimension
    x_starts = calculate_patch_starts(m, patch_size)
    y_starts = calculate_patch_starts(n, patch_size)
    z_starts = calculate_patch_starts(l, patch_size)
    
    # Extract patches from each array
    for arr in arrays:
        for x in x_starts:
            for y in y_starts:
                for z in z_starts:
                    patch = arr[
                        x:x + patch_size,
                        y:y + patch_size,
                        z:z + patch_size
                    ]
                    patches.append(patch)
                    coordinates.append((x, y, z))
    
    return patches, coordinates
def reconstruct_array(patches: List[np.ndarray], 
                     coordinates: List[Tuple[int, int, int]], 
                     original_shape: Tuple[int, int, int]) -> np.ndarray:
    reconstructed = np.zeros(original_shape, dtype=np.int64)  # To track overlapping regions
    
    patch_size = patches[0].shape[0]
    
    for patch, (x, y, z) in zip(patches, coordinates):
        reconstructed[
            x:x + patch_size,
            y:y + patch_size,
            z:z + patch_size
        ] = patch
        
    
    return reconstructed
def calculate_patch_starts(dimension_size: int, patch_size: int) -> List[int]:
    if dimension_size <= patch_size:
        return [0]
        
    # Calculate number of patches needed
    n_patches = np.ceil(dimension_size / patch_size)
    
    if n_patches == 1:
        return [0]
    
    # Calculate overlap
    total_overlap = (n_patches * patch_size - dimension_size) / (n_patches - 1)
    
    # Generate starting positions
    positions = []
    for i in range(int(n_patches)):
        pos = int(i * (patch_size - total_overlap))
        if pos + patch_size > dimension_size:
            pos = dimension_size - patch_size
        if pos not in positions:  # Avoid duplicates
            positions.append(pos)
    
    return positions
import pandas as pd

def dict_to_df(coord_dict, experiment_name):
    # Create lists to store data
    all_coords = []
    all_labels = []
    
    # Process each label and its coordinates
    for label, coords in coord_dict.items():
        all_coords.append(coords)
        all_labels.extend([label] * len(coords))
    
    # Concatenate all coordinates
    all_coords = np.vstack(all_coords)
    
    df = pd.DataFrame({
        'experiment': experiment_name,
        'particle_type': all_labels,
        'x': all_coords[:, 0],
        'y': all_coords[:, 1],
        'z': all_coords[:, 2]
    })

    
    return df

TRAIN_DATA_DIR = "/kaggle/input/create-numpy-dataset-exp-name"
import json
copick_config_path = TRAIN_DATA_DIR + "/copick.config"

with open(copick_config_path) as f:
    copick_config = json.load(f)

copick_config['static_root'] = '/kaggle/input/czii-cryo-et-object-identification/test/static'

copick_test_config_path = 'copick_test.config'

with open(copick_test_config_path, 'w') as outfile:
    json.dump(copick_config, outfile)
import copick

root = copick.from_file(copick_test_config_path)

copick_user_name = "copickUtils"
copick_segmentation_name = "paintedPicks"
voxel_size = 11
tomo_type = "denoised"
inference_transforms = Compose([
    EnsureChannelFirstd(keys=["image"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image"], axcodes="RAS")
])

id_to_name = {1: "apo-ferritin", 
              2: "beta-amylase",
              3: "beta-galactosidase", 
              4: "ribosome", 
              5: "thyroglobulin", 
              6: "virus-like-particle"}

##############################################################################################################################
##############################################################################################################################
BLOB_THRESHOLD = 140
CERTAINTY_THRESHOLD = 0.035
#BLOB_THRESHOLD = 255
#CERTAINTY_THRESHOLD = 0.05
##############################################################################################################################
##############################################################################################################################

classes = [1, 2, 3, 4, 5, 6]

def load_models(model_paths):
    models = []
    for model_path in model_paths:
        channels = (48, 64, 80, 80)
        strides_pattern = (2, 2, 1)       
        num_res_units = 1
        learning_rate = 1e-3
        num_epochs = 100
        model = Model(channels=channels, strides=strides_pattern, num_res_units=num_res_units)
        
        weights =torch.load(model_path)['state_dict']
        model.load_state_dict(weights)
        model.to('cuda')
        model.eval()
        models.append(model)
    return models


model_paths = [
    '/kaggle/input/cziials-a-230-unet/UNet-Model-val_metric0.450.ckpt',
]


models = load_models(model_paths)
def ensemble_prediction_tta(models, input_tensor, threshold=0.5):
    probs_list = []
    data_copy0 = input_tensor.clone()
    data_copy0=torch.flip(data_copy0, dims=[2])
    data_copy1 = input_tensor.clone()
    data_copy1=torch.flip(data_copy1, dims=[3])
    data_copy2 = input_tensor.clone()
    data_copy2=torch.flip(data_copy2, dims=[4])
    data_copy3 = input_tensor.clone()
    data_copy3 = data_copy3.rot90(1, dims=[3, 4])
    with torch.no_grad():
        model_output0 = model(input_tensor)
        model_output1 = model(data_copy0)
        model_output1=torch.flip(model_output1, dims=[2])
        model_output2 = model(data_copy1)
        model_output2=torch.flip(model_output2, dims=[3])
        model_output3 = model(data_copy2)
        model_output3=torch.flip(model_output3, dims=[4])
        probs0 = torch.softmax(model_output0[0], dim=0)
        probs1 = torch.softmax(model_output1[0], dim=0)
        probs2 = torch.softmax(model_output2[0], dim=0)
        probs3 = torch.softmax(model_output3[0], dim=0)
        probs_list.append(probs0)
        probs_list.append(probs1)
        probs_list.append(probs2)
        probs_list.append(probs3)
    avg_probs = torch.mean(torch.stack(probs_list), dim=0)
    thresh_probs = avg_probs > threshold
    _, max_classes = thresh_probs.max(dim=0)
    return max_classes
sub=[]
for model in models:
    with torch.no_grad():
        location_df = []
        for run in root.runs:
            tomo = run.get_voxel_spacing(10)
            tomo = tomo.get_tomogram(tomo_type).numpy()
            tomo_patches, coordinates = extract_3d_patches_minimal_overlap([tomo], 96)
            tomo_patched_data = [{"image": img} for img in tomo_patches]
            tomo_ds = CacheDataset(data=tomo_patched_data, transform=inference_transforms, cache_rate=1.0)
            pred_masks = []
            for i in tqdm(range(len(tomo_ds))):
                input_tensor = tomo_ds[i]['image'].unsqueeze(0).to("cuda")
                max_classes = ensemble_prediction_tta(models, input_tensor, threshold=CERTAINTY_THRESHOLD)
                pred_masks.append(max_classes.cpu().numpy())
            reconstructed_mask = reconstruct_array(pred_masks, coordinates, tomo.shape)
            location = {}
            for c in classes:
                cc = cc3d.connected_components(reconstructed_mask == c)
                stats = cc3d.statistics(cc)
                zyx = stats['centroids'][1:] * 10.012444  # 转换单位
                zyx_large = zyx[stats['voxel_counts'][1:] > BLOB_THRESHOLD]
                xyz = np.ascontiguousarray(zyx_large[:, ::-1])
                location[id_to_name[c]] = xyz
            df = dict_to_df(location, run.name)
            location_df.append(df)
        location_df = pd.concat(location_df)
        location_df.insert(loc=0, column='id', value=np.arange(len(location_df)))

<ipython-input-12-4cb51b5d90db>:142: DeprecationWarning: config_type not found in config file, defaulting to filesystem
  root = copick.from_file(copick_test_config_path)
100%|██████████| 98/98 [00:12<00:00,  8.10it/s]


In [13]:
location_df.tail()

,id,experiment,particle_type,x,y,z
323,884,TS_6_4,virus-like-particle,126.022125,5623.885237,1053.390669
324,885,TS_6_4,virus-like-particle,1455.693728,4808.580410,1114.245736
325,886,TS_6_4,virus-like-particle,4516.127768,5150.053215,1164.027166
326,887,TS_6_4,virus-like-particle,3109.298117,782.166881,1243.345624
327,888,TS_6_4,virus-like-particle,2393.727994,5138.904552,1515.507083


# **《《《 Blend Detection Models 》》》**

In [14]:
submission_['weight'] = 1
submission_['model_type'] = 'yolo'   
location_df['weight'] = 1
location_df['model_type'] = 'unet'

df = pd.concat([submission_,location_df], ignore_index=True)

particle_names = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']
particle_radius = {
    'apo-ferritin': 60,
    'beta-amylase': 65,
    'beta-galactosidase': 90,
    'ribosome': 150,
    'thyroglobulin': 130,
    'virus-like-particle': 135,
}

final = []
for pidx, p in enumerate(particle_names):
    '''The goal here is to keep all predictions, 
    but assign a weight of 1 to single-model predictions, 2 to 2-model predictions'''
    pdf = df[df['particle_type'] == p].reset_index(drop=True)
    p_rad = particle_radius[p]
    
    grouped = pdf.groupby(['experiment'])

    
    for exp, group in grouped:
        group = group.reset_index(drop=True)
        coords = group[['x', 'y', 'z']].values
        db = DBSCAN(eps=p_rad, min_samples=2, metric='euclidean').fit(coords)
        labels = db.labels_
        group['cluster'] = labels
        
        for cluster_id in np.unique(labels):
            if cluster_id == -1:
                continue
            
            cluster_points = group[group['cluster'] == cluster_id]
            unique_model_types = cluster_points['model_type'].nunique()
            
            if len(cluster_points) > 1:
                if unique_model_types >= 2:
                    group.loc[group['cluster'] == cluster_id, 'weight'] = 2  #so we still need one more model
                
                avg_x = cluster_points['x'].mean()
                avg_y = cluster_points['y'].mean()
                avg_z = cluster_points['z'].mean()
                
                group.loc[group['cluster'] == cluster_id, ['x', 'y', 'z']] = avg_x, avg_y, avg_z
        
        # Check for deduplication effect
        group = group.drop_duplicates(subset=['x', 'y', 'z'])
        final.append(group)

df_save = pd.concat(final, ignore_index=True)
df_save = df_save.drop(columns=['cluster'])
df_roi = df_save.sort_values(by=['experiment', 'particle_type']).reset_index(drop=True)
df_roi['id'] = np.arange(0, len(df_roi))

## **《《《 Reclassify ROIs 》》》**

In [15]:
df_roi.tail()

,id,experiment,particle_type,x,y,z,weight,model_type
1295,1295,TS_6_4,virus-like-particle,6003.722744,2290.265819,712.909502,1,unet
1296,1296,TS_6_4,virus-like-particle,6112.628157,2456.344802,696.269087,1,unet
1297,1297,TS_6_4,virus-like-particle,4420.367819,96.296153,909.155157,1,unet
1298,1298,TS_6_4,virus-like-particle,3836.956875,3585.742541,1005.818223,1,unet
1299,1299,TS_6_4,virus-like-particle,3109.298117,782.166881,1243.345624,1,unet


In [16]:
class InferConfig:
    '''Wrapper class for inference & model configuration'''
    def __init__(self):
        self.EXTRA_CORES = None #None will result in the max cpu count used.
        self.BATCH_SIZE = 16
        self.ROI_SIZE = 32
        self.ZARR_SCALE = 10.012444
        self.CLASS_NAMES = ['apo-ferritin',
                            'beta-amylase',
                            'beta-galactosidase',
                            'empty',
                            'ribosome',
                            'thyroglobulin',
                            'virus-like-particle']
        self.MODELS_OVER_THRESHOLD = 3

class ImageConfig:
    '''Wrapper class for image processing parameters'''
    INPUT_MEAN = [ 0.485, 0.456, 0.406 ] #values from ImageNet.
    INPUT_STD = [ 0.229, 0.224, 0.225 ] #values from ImageNet.

#Just commenting out most of the models so no need to make them all public, but allow the notebook to run
MODELS = [ #{'backbone': 'eca_nfnet_l0.ra2_in1k',  #mean CV: 0.955,  mean - sd = 0.936
           #  'weights': '/kaggle/input/czii_exp_24_ecanfnet_2sl/pytorch/default/1/Run_01_best_weights.pt',
           #  'activation': 'sigmoid',
           #  'slices': (np.arange(8, 32, 2),0),  #(np.arange(10,22), 0), (np.arange(8, 32, 2),0)
           #  'thresholds': {'apo-ferritin': 0.36,
           #       'beta-amylase': 0.36,
           #       'beta-galactosidase': 0.36,
           #       'empty': 0.6,
           #       'ribosome': 0.36,
           #       'thyroglobulin' : 0.36,
           #       'virus-like-particle': 0.36},
           #  'tta': [0,1,2]
           # },

           # {'backbone': 'efficientvit_b3.r288_in1k',    #mean CV:  0.946,  mean-sd: 0.938
           #  'weights': '/kaggle/input/czii_exp_22_efficientvit_2sl/pytorch/default/1/Run_01_best_weights.pt',
           #  'activation': 'sigmoid',
           #  'slices': (np.arange(8, 32, 2),0),  #(np.arange(10,22), 0), (np.arange(8, 32, 2),0)
           #  'thresholds': {'apo-ferritin': 0.45,
           #       'beta-amylase': 0.45,
           #       'beta-galactosidase': 0.45,
           #       'empty': 0.6,
           #       'ribosome': 0.45,
           #       'thyroglobulin' : 0.45,
           #       'virus-like-particle': 0.45},
           #  'tta': [2,3,4]
           # },
          
            {'backbone': 'mobilevit_xs.cvnets_in1k',   
             'weights': '/kaggle/input/czii_exp_21_mobilevitxs_2sl/pytorch/default/1/Run_01_best_weights.pt',  #7 classes, mean 0.961,  mean-sd  0.947
            # 'weights': '/kaggle/input/czii_exp_29_mobilevitxs_6_2sl/pytorch/default/1/Run_01_best_weights.pt', # 6 class, mean 0.964,  0.947
             'activation': 'sigmoid',
             'slices': (np.arange(8, 32, 2),0),  #(np.arange(10,22), 0), (np.arange(8, 32, 2),0)
             'thresholds': {'apo-ferritin': 0.36,
                  'beta-amylase': 0.36,
                  'beta-galactosidase': 0.36,
                  'empty': 0.6,  #remove for 6-class models
                  'ribosome': 0.36,
                  'thyroglobulin' : 0.36,
                  'virus-like-particle': 0.36},
             'tta': [4,5,0]
            },


            #{'backbone': 'mobilevitv2_175.cvnets_in22k_ft_in1k_384',     #0.944,  0.933
            # 'weights': '/kaggle/input/czii_exp_26_mobilevit2_2sl/pytorch/default/1/Run_01_best_weights.pt',
            # 'activation': 'sigmoid',
            # 'slices': (np.arange(8, 32, 2),0),  #(np.arange(10,22), 0), (np.arange(8, 32, 2),0)
            # 'thresholds': {'apo-ferritin': 0.45,
            #      'beta-amylase': 0.45,
            #      'beta-galactosidase': 0.45,
            #      'empty': 0.6,    #remove for 6-class models
            #      'ribosome': 0.45,
            #      'thyroglobulin' : 0.45,
            #      'virus-like-particle': 0.45},
            # 'tta': [5,0,1]
            #},


            ### Older ones, before I started randomising more layers for pre-train

            #Tried dropping this, but the score went backwards.  Have a go at imroving it.
            #{'backbone': 'mobilevit_s.cvnets_in1k', #Exp_10   MobileViT  Central 12 slices  
            # 'weights': '/kaggle/input/czii_exp_10_run_03_mobilevit/pytorch/default/1/Run_03_best_weights.pt',   #.948, 0.928 (7 classes)
             #'weights': '/kaggle/input/czii_exp_30_mobilevits_6_1sl/pytorch/default/1/Run_01_best_weights.pt',   #.953, 0.934 (6 classes)
            # 'activation': 'sigmoid',
            # 'slices': (np.arange(10,22), 0), #(np.arange(10,22), 0), (np.arange(8, 32, 2),0)
            # 'thresholds': {'apo-ferritin': 0.45,
            #      'beta-amylase': 0.45,
            #      'beta-galactosidase': 0.45,
            #      'empty': 0.6,
            #      'ribosome': 0.45,
            #      'thyroglobulin' : 0.45,
            #      'virus-like-particle': 0.45},
            # 'tta': [0,1,2]
            #},

            #{'backbone': 'mobilevit_xs.cvnets_in1k',  #Exp_14  mobilevit_xs   12 central slices   # 0.958,  0.943
            # 'weights':'/kaggle/input/czii_exp_14_run_02/pytorch/default/1/Run_02_best_weights.pt',
            # 'activation': 'sigmoid',
            # 'slices': (np.arange(10,22), 0), #(np.arange(10,22), 0), (np.arange(8, 32, 2),0)
            # 'thresholds': {'apo-ferritin': 0.45,
             #     'beta-amylase': 0.45,
           #      'beta-galactosidase': 0.45,
            #      'empty': 0.6,
            #      'ribosome': 0.45,
            #      'thyroglobulin' : 0.45,
            #      'virus-like-particle': 0.45},
            # 'tta': [1,2,3]
            #},

            #{'backbone': 'mobilevit_s.cvnets_in1k',  #Exp_09   MobileViT Every second slice    #0.954  0.944
            #             'weights': '/kaggle/input/czii_exp_00_run_02_mobilevit/pytorch/default/1/Run_02_best_weights.pt',
            # 'activation': 'sigmoid',
            # 'slices': (np.arange(8, 32, 2),0), #(np.arange(10,22), 0),
            # 'thresholds': {'apo-ferritin': 0.45,
            #      'beta-amylase': 0.45,
            #      'beta-galactosidase': 0.45,
            #      'empty': 0.6,
            #      'ribosome': 0.45,
            #      'thyroglobulin' : 0.45,
            #      'virus-like-particle': 0.45},
            # 'tta': [2,3,4]
            #},
         ]

In [17]:
def get_model(kwargs, gpu_idx):
    backbone = kwargs['backbone']
    weights_path = kwargs['weights']   
    #device = torch.device(f"cuda:{kwargs['gpu']}")
    ckpt = torch.load(weights_path)#["model_state_dict"]
    model = TomoModel(model_name=backbone, num_classes = len(kwargs['thresholds']))
    model.load_state_dict(ckpt) #["model_state_dict"]
    model.eval()
    model = model.to(f'cuda:{gpu_idx}')
    #model = torch.nn.DataParallel(model, device_ids = [0,1]).to(torch.device('cuda'))
    return model


class Augmentation():
    '''Vol Augmentation to work with 3d tensors Outputting (z,y,x)
       Img Augmentation to work with numpy arrays, initial shape (20,20,3) or similar
       Outputing (3,16,16) as a PyTorch Tensor (C,H,W)'''

    def _slice_one_dim(self, array, slice_pattern):
        """
        Apply a slice or indices dynamically to the first or second dimension of a 3D array.
        slice_pattern is a tuple (the pattern (array, list of slice), the dimension (0 or 1)
        """
        pattern= slice_pattern[0]
        dim = slice_pattern[1]
        if isinstance(pattern, (slice, list, np.ndarray)):
            if dim==0:
                return array[pattern]
            else: 
                return array[:, pattern]
        else:
            raise ValueError("Slice pattern must be a slice, list, or numpy array.")
    
    def __init__(self, mean, std, height, width, layer_pattern):
        self.mean = mean
        self.std = std

        self.vol_ttas = [
                         Compose([Lambda(func = lambda x: self._slice_one_dim(x, layer_pattern)),
                                RandSpatialCrop(roi_size=(32, 32), random_center=False, random_size=False)
                                ]),
                        Compose([
                                RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                Lambda(func = lambda x: self._slice_one_dim(x, layer_pattern)),
                                RandSpatialCrop(roi_size=(32, 32), random_center=False, random_size=False)
                                ]),
                        Compose([
                                RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                Lambda(func = lambda x: self._slice_one_dim(x, layer_pattern)),
                                RandSpatialCrop(roi_size=(32, 32), random_center=False, random_size=False)
                                ]),
                       Compose([
                                RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                Lambda(func = lambda x: self._slice_one_dim(x, layer_pattern)),
                                RandSpatialCrop(roi_size=(32, 32), random_center=False, random_size=False)
                                ]),
                        Compose([
                                 RandFlip(prob=1, spatial_axis=0),
                                 RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                 Lambda(func = lambda x: self._slice_one_dim(x, layer_pattern)),
                                 RandSpatialCrop(roi_size=(32, 32), random_center=False, random_size=False)
                                ]),
                        Compose([
                                 RandFlip(prob=1, spatial_axis=1),
                                 RandRotate90(prob=1, spatial_axes=[0, 1], max_k=1),
                                 Lambda(func = lambda x: self._slice_one_dim(x, layer_pattern)),
                                 RandSpatialCrop(roi_size=(32, 32), random_center=False, random_size=False)
                                ])
                        ]

        self.img_val = A.Compose([
            A.CenterCrop(height=2*height, width=2*width, p=1),
            A.Normalize(mean=self.mean, std=self.std, max_pixel_value=1), 
            ToTensorV2()]) #Note: ToTensorV2 turns HWC Numpy to CHW Tensor


class TomoDataset(Dataset):
    def __init__(self,
                 df,
                 roi_dict,
                 vol_transform=None,
                 img_transform=None):
        self.df = df
        self.vol_transform = vol_transform
        self.img_transform = img_transform
        self.roi_dict = roi_dict
        
    def __len__(self):
        return len(self.df)


    def reshape_tomo(self, array):
        """
        Starting with an array shaped (z, y, x) (Slices, Height, Width):
        - Crop so the total slices is exactly 12.
        - Break into 4 parts along the slice direction.
        - Re-assemble the 4 parts so the assembled array is (3, 32, 32).
        - Transpose, so it is left as (Height, Width, Channels) to be treated as an image.
        """

        z_slices = array.shape[0]
        assert z_slices % 4 == 0, "The number of slices after cropping must be divisible by 4"
        parts = np.array_split(array, 4, axis=0)
        top = np.concatenate(parts[:2], axis=2)  # Concatenate the first two along width
        bottom = np.concatenate(parts[2:], axis=2)  # Concatenate the last two along width
        assembled_array = np.concatenate([top, bottom], axis=1)  # Concatenate along height
        hwc_array = np.transpose(assembled_array, (1, 2, 0))  # Transpose to (y, x, z)
        return hwc_array


    def __getitem__(self, index):
        row = self.df.iloc[index]
        id = row['id']
        tomo = self.roi_dict[id]
        tomo = self.vol_transform(tomo)
        image = self.reshape_tomo(tomo) #Numpy array like an image, of shape (H, W, 3)
        min_vals = np.min(image)
        max_vals = np.max(image) 
        image = (image - min_vals) / (max_vals - min_vals)  #Min-Max normalise on [0,1]
        image_tensor = self.img_transform(image=image)['image']  #Tensor with channels first (3, H, W)
        return image_tensor


class BasicHead(nn.Module):
    '''Bare bones fc classifier head'''
    def __init__(self, in_features, num_classes):
        super(BasicHead, self).__init__()
        self.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.fc(x)
        return x


class TomoModel(pl.LightningModule):
    '''Pytorch Lightning Model'''
    def __init__(self,
                model_name = 'resnet18.a1_in1k',
                num_classes = 7):
        super().__init__()

        self.num_classes = num_classes
        self.backbone = timm.create_model(model_name, pretrained=False)

        #varios logic to replace the classifier head depending on model architecture
        if model_name.startswith('eca_nfnet') or model_name.startswith('convnext'):
            self.in_features = self.backbone.head.fc.in_features
            setattr(self.backbone.head, 'fc', BasicHead(self.in_features, self.num_classes))
        elif model_name.startswith('efficientvit'): #or model_name.startswith(deit_tiny):
            for layer in self.backbone.head.classifier:
                if isinstance(layer, torch.nn.Linear):
                    self.in_features = layer.in_features
                    break
            setattr(self.backbone.head, 'classifier', BasicHead(self.in_features, self.num_classes))
        elif model_name.startswith('mobilevit'):
            self.in_features = self.backbone.head.fc.in_features
            setattr(self.backbone.head, 'fc', BasicHead(self.in_features, self.num_classes))
        elif model_name.startswith('deit_tiny'):
            self.in_features = self.backbone.head.in_features
            setattr(self.backbone.head, 'linear', BasicHead(self.in_features, self.num_classes))
        elif hasattr(self.backbone, 'fc'):
            self.in_features = self.backbone.fc.in_features
            setattr(self.backbone, 'fc', BasicHead(self.in_features, self.num_classes))
        elif hasattr(self.backbone, 'classifier'):
            self.in_features = self.backbone.classifier.in_features
            setattr(self.backbone, 'classifier', BasicHead(self.in_features, self.num_classes))
        elif hasattr(self.backbone, 'head') and hasattr(self.backbone.head, 'linear'):
            self.in_features = self.backbone.head.linear.in_features
            setattr(self.backbone, 'linear', BasicHead(self.in_features, self.num_classes))
        else:
            raise AttributeError("The backbone does not have a 'fc' or 'classifier' or 'head', layer. Update the code to handle the correct layer name.")

    def forward(self, images):
        logits = self.backbone(images)
        return logits


def crop_roi(tomo, row):
    slicing_cols = ['object_type','x_start', 'x_end', 'y_start', 'y_end', 'z_start', 'z_end']
    records = group[slicing_cols].to_dict(orient='records')
    
    for idx, row in enumerate(records):
        crop = tomo[0][row['z_start']:row['z_end'],
                       row['y_start']:row['y_end'],
                       row['x_start']:row['x_end']]
        type=row['object_type']
        np.save(paths.DESTN_FLDRS[type] / f'{sample}_{idx}.npy', crop)


def preprocess_rois(df, tomo, n_jobs=-1):
    """
    Extract Regions of Interest (ROIs) from a tomograph and store them in a dictionary.

    Args:
        df (pd.DataFrame): DataFrame containing slicing information for ROIs.
        folder_pth (Path): Path to the folder containing the tomograph data.
        sample_name (str): Name of the sample to process.
        size (tuple): Size of the ROI (not directly used but kept for flexibility).
        n_jobs (int): Number of parallel jobs. Defaults to -1 (use all available CPUs).

    Returns:
        dict: Dictionary of ROIs keyed by their 'id'.
    """

    slicing_cols = ['id', 'x_start', 'x_end', 'y_start', 'y_end', 'z_start', 'z_end']
    records = df[slicing_cols].to_dict(orient='records')

    def extract_roi(row):
        crop = tomo[row['z_start']:row['z_end'],
                    row['y_start']:row['y_end'],
                    row['x_start']:row['x_end']]
        return row['id'], crop

    roi_list = Parallel(n_jobs=n_jobs)(
        delayed(extract_roi)(row) for row in records)
    roi_dict = {roi_id: crop for roi_id, crop in roi_list}

    return roi_dict


def open_tomo(experiment, test_folder):
        zarr_path = test_folder / experiment / 'VoxelSpacing10.000/denoised.zarr'
        volume = zarr.open(zarr_path, mode='r')[0]  # Access the highest resolution tomograph
        lower, upper = np.percentile(volume, (0.5, 99.5))
        clipped = np.clip(volume, lower, upper)
        scaled = ((clipped - lower) / (upper - lower + 1e-12) * 255).astype(np.uint8)
        return scaled


def predict_with_one_model(experiment, df, model_tuple, tomo, batch_size=32, num_workers=4):
    model = model_tuple[0] # adjust this when I have more models
    model_args = model_tuple[1]
    thresholds = model_args['thresholds']
    activation = model_args['activation']
    tta_choices = model_args['tta']  #A list of integers
    test_folder = Path('/kaggle/input/czii-cryo-et-object-identification/test/static/ExperimentRuns/')
    class_names = {0: "apo-ferritin", 
                   1: "beta-amylase",
                   2: "beta-galactosidase",
                   3: "empty",
                   4: "ribosome", 
                   5: "thyroglobulin", 
                   6: "virus-like-particle"}

    
    rois = preprocess_rois(df, tomo)
    
    transforms = Augmentation(mean = [ 0.485, 0.456, 0.406 ],
                              std  = [ 0.229, 0.224, 0.225 ],
                              height = 32,
                              width = 32,
                              layer_pattern = model_args['slices'])    # mean, std, height, width, layer_pattern

    tta_dfs = []
    for tta in tta_choices:
        ds = TomoDataset(df, rois, transforms.vol_ttas[tta], transforms.img_val)
        print(f'inferring experiment {experiment}')
        
        predictions = []
        loader = DataLoader(ds, 
                        batch_size=batch_size, 
                        persistent_workers=True,
                        num_workers = num_workers, 
                        shuffle=False,
                        drop_last=False
                        )
        device = model.device
        with torch.no_grad():
            for images in loader:
                images = images.to(device)
                logits = model(images)
                if activation == 'Softmax':
                    probs = F.softmax(logits, dim=1).detach().cpu().numpy()  #batch_size x num_classes
                else:
                    probs = F.sigmoid(logits).detach().cpu().numpy()  #batch_size x num_classes
                predictions.append(probs)
        all_predictions = np.vstack(predictions)  # Shape: (total_samples x num_classes)

        if all_predictions.shape[1] == 6:
            all_predictions = np.insert(all_predictions, 3, 0, axis=1)  #Add 0's for a possible 'empty' column for 6 class models.
    
        predictions_df = pd.DataFrame(
            all_predictions, 
            columns=[class_names[i] for i in range(all_predictions.shape[1])]
        )
    
        cols_to_keep = ['id', 'experiment',	'particle_type', 'x', 'y', 'z', 'weight']
        _df = df[cols_to_keep].reset_index(drop=True)
        tta_df = pd.concat([_df, predictions_df], axis=1)
        tta_dfs.append(tta_df)

    pred_cols = [val for val in class_names.values()]
    stacked_values = np.stack([df[pred_cols].values for df in tta_dfs])
    #mean_values = np.mean(stacked_values, axis=0)
    combined_values = np.mean(stacked_values, axis=0)
    #max_values = np.max(stacked_values, axis=0)
    #combined_values = (mean_values + max_values) / 2

    df_model = tta_dfs[0].copy()
    df_model[pred_cols] = combined_values

    return df_model


def predict_one_sample(experiment, df, model_tuples, tomo, batch_size=32, num_workers=2):

    model_dataframes = []
    for model_tuple in model_tuples:
        model_df = predict_with_one_model(experiment, df, model_tuple, tomo,batch_size, num_workers)
        model_dataframes.append(model_df)

    text_columns = ['id', 'x', 'y', 'z', 'experiment', 'particle_type', 'weight']  # Columns to leave unchanged
    numeric_columns = model_dataframes[0].columns.difference(text_columns).tolist()  #Columns to sum
    #numeric_values = np.sum([df[numeric_columns].values for df in model_dataframes], axis=0)
    numeric_values = np.mean([df[numeric_columns].values for df in model_dataframes], axis=0)


    
    numeric_df = pd.DataFrame(numeric_values, columns=numeric_columns)
    text_df = model_dataframes[0][text_columns]
    result = pd.concat([text_df, numeric_df], axis=1)
    result = result[model_dataframes[0].columns]

    thresholds = model_tuples[0][1]['thresholds']  #just grabbing the thresholds from the first model in the list
    for class_name, threshold in thresholds.items():
        original_values = result[class_name]  # Preserve the original values
        result[class_name] = (original_values >= threshold).astype(int)  # 0 or 1
        result[class_name] += (original_values >= threshold + 0.19).astype(int)  # 0, 1, or 2
        #result[class_name] += (original_values >= threshold + 0.64).astype(int)  # 0, 1, 2, or 3  #seems unhelpful
    return result


In [18]:
def infer_one_half(df, infer_cfg, image_cfg, gpu_idx):
    models = [(get_model(args_dict, gpu_idx),args_dict) for args_dict in MODELS] #note, the args_dict may be needed for pre-processing
    test_folder = Path('/kaggle/input/czii-cryo-et-object-identification/test/static/ExperimentRuns/')
    preds = {}
    
    for experiment, sample_df in df.groupby(by='experiment'):
        tomo = open_tomo(experiment, test_folder)
        preds[experiment] = predict_one_sample(experiment, 
                                               sample_df, 
                                               models, 
                                               tomo, 
                                               batch_size=32)
    return preds

def parallel_inference(df, infer_cfg, image_cfg, gpus=[0, 1]):
    # Split the DataFrame into two halves
    df_0, df_1 = split_dataframe(df)
    
    with ProcessPoolExecutor(max_workers=2) as executor:
        results = list(executor.map(infer_one_half, 
                                    [df_0, df_1], 
                                    [infer_cfg, infer_cfg], 
                                    [image_cfg, image_cfg], 
                                    gpus))
    
    # Merge the results from both halves (use dict union or any other method)
    all_preds = all_preds_list[0] | all_preds_list[1]
    
    return all_preds

def split_dataframe(df):
    # Split DataFrame into two nearly equal parts
    middle_index = len(df) // 2
    df_0 = df.iloc[:middle_index]
    df_1 = df.iloc[middle_index:]
    return df_0, df_1

In [19]:
if __name__ == '__main__':
    infer_cfg = InferConfig()
    image_cfg = ImageConfig()
    
    HALF = infer_cfg.ROI_SIZE//2
    roi_df = df_roi.copy()
    roi_df['x_v'] = round(roi_df['x'] / infer_cfg.ZARR_SCALE).astype(int).clip(lower=HALF, upper=630-HALF) # 8 - 622
    roi_df['y_v'] = round(roi_df['y'] / infer_cfg.ZARR_SCALE).astype(int).clip(lower=HALF, upper=630-HALF) # 8 - 622
    roi_df['z_v'] = round(roi_df['z'] / infer_cfg.ZARR_SCALE).astype(int).clip(lower=HALF, upper=184-HALF) # 8 - 176
    roi_df['x_start'], roi_df['x_end'] = roi_df['x_v'] - HALF, roi_df['x_v'] + HALF
    roi_df['y_start'], roi_df['y_end'] = roi_df['y_v'] - HALF, roi_df['y_v'] + HALF
    roi_df['z_start'], roi_df['z_end'] = roi_df['z_v'] - HALF, roi_df['z_v'] + HALF

    #all_preds = parallel_inference(roi_df, infer_cfg, image_cfg, [0,1])  #Locks up.  Haven't figured out why.
    all_preds = infer_one_half(roi_df, infer_cfg, image_cfg, 0)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


inferring experiment TS_5_4


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


inferring experiment TS_5_4


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


inferring experiment TS_6_4


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


inferring experiment TS_6_4


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [20]:
results_df = pd.concat(all_preds.values(), ignore_index=True)
results_df.tail()

,id,experiment,particle_type,x,y,z,weight,apo-ferritin,beta-amylase,beta-galactosidase,empty,ribosome,thyroglobulin,virus-like-particle
1295,1295,TS_6_4,virus-like-particle,6003.722744,2290.265819,712.909502,1,0,0,0,0,0,0,1
1296,1296,TS_6_4,virus-like-particle,6112.628157,2456.344802,696.269087,1,0,0,0,0,0,0,2
1297,1297,TS_6_4,virus-like-particle,4420.367819,96.296153,909.155157,1,0,0,0,0,1,0,2
1298,1298,TS_6_4,virus-like-particle,3836.956875,3585.742541,1005.818223,1,0,0,0,0,0,0,2
1299,1299,TS_6_4,virus-like-particle,3109.298117,782.166881,1243.345624,1,0,0,0,0,1,1,1


In [21]:
results_df['virus-like-particle'].value_counts()

virus-like-particle
0    1223
2      51
1      26
Name: count, dtype: int64

In [22]:
results_df['beta-amylase'].value_counts()

beta-amylase
0    982
2    167
1    151
Name: count, dtype: int64

In [23]:
results_df['thyroglobulin'].value_counts()

thyroglobulin
0    785
2    354
1    161
Name: count, dtype: int64

In [24]:
results_df['empty'].value_counts()

empty
0    1237
1      40
2      23
Name: count, dtype: int64

In [25]:
results_df['beta-amylase'] = 0
results_df = results_df[~(results_df[infer_cfg.CLASS_NAMES] == 0).all(axis=1)]  #remove rows where all reclassify predictions == 0


In [26]:
results_df['empty'].value_counts()

empty
0    1124
1      40
2      23
Name: count, dtype: int64

In [27]:
easy = ["apo-ferritin",  "ribosome", "virus-like-particle"]
hard = ["beta-galactosidase", "thyroglobulin"]
certainly_easy = 2
mask = results_df[easy].ge(certainly_easy).any(axis=1)
results_df.loc[mask, hard] = 0

In [28]:
scored_names = ["apo-ferritin", 
                "beta-galactosidase",
                "ribosome", 
                "thyroglobulin", 
                "virus-like-particle"]

ohe = pd.get_dummies(results_df['particle_type']).reindex(columns=scored_names, fill_value=0)
ohe[scored_names] = ohe[scored_names].mul(results_df['weight'], axis=0)
results_df[scored_names] += ohe
results_df.head()

,id,experiment,particle_type,x,y,z,weight,apo-ferritin,beta-amylase,beta-galactosidase,empty,ribosome,thyroglobulin,virus-like-particle
0,0,TS_5_4,apo-ferritin,4985.038208,4833.595703,20.000000,1,1,0,0,0,0,1,0
1,1,TS_5_4,apo-ferritin,5873.572518,5136.602123,81.830725,2,4,0,0,0,0,0,0
2,2,TS_5_4,apo-ferritin,5712.765924,4998.938474,116.047437,2,4,0,0,0,0,0,0
3,3,TS_5_4,apo-ferritin,5747.706355,5110.587367,86.628920,2,4,0,0,0,0,0,0
4,4,TS_5_4,apo-ferritin,5473.405809,1520.826088,84.250002,2,3,0,0,0,0,0,0


In [29]:
keep_only_top = False

if keep_only_top:
    results_df['particle_type'] = results_df.iloc[:, -7:].idxmax(axis=1)
else:
    cols_to_keep = ['experiment', 'x', 'y', 'z'] #+ infer_cfg.CLASS_NAMES  #For debugging/analysis
    positive_rows = []
    for _, row in results_df.iterrows():
        for column_name in scored_names:
            if row[column_name] >= infer_cfg.MODELS_OVER_THRESHOLD:  #At least this many models over threshold to predict a positive.
                new_row = {col: row[col] for col in cols_to_keep}
                new_row['particle_type'] = column_name
                positive_rows.append(new_row)
    results_df = pd.DataFrame(positive_rows)

results_df = results_df.reset_index(drop=True)
results_df['id'] = results_df.index
results_df.head()

,experiment,x,y,z,particle_type,id
0,TS_5_4,5873.572518,5136.602123,81.830725,apo-ferritin,0
1,TS_5_4,5712.765924,4998.938474,116.047437,apo-ferritin,1
2,TS_5_4,5747.706355,5110.587367,86.628920,apo-ferritin,2
3,TS_5_4,5473.405809,1520.826088,84.250002,apo-ferritin,3
4,TS_5_4,5294.213770,4177.939746,135.000000,apo-ferritin,4


In [30]:
cols_to_keep = ['id','experiment', 'particle_type', 'x', 'y', 'z']
results_df = results_df[cols_to_keep]
results_df.tail()

,id,experiment,particle_type,x,y,z
898,898,TS_6_4,virus-like-particle,2402.670317,5140.823583,1522.753541
899,899,TS_6_4,virus-like-particle,3625.590307,3705.609113,511.765081
900,900,TS_6_4,virus-like-particle,6112.628157,2456.344802,696.269087
901,901,TS_6_4,virus-like-particle,4420.367819,96.296153,909.155157
902,902,TS_6_4,virus-like-particle,3836.956875,3585.742541,1005.818223


In [31]:
results_df.to_csv("submission.csv", index=False)